

#  ***insert all pyspark liberties into this notebook***



In [65]:


import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [13]:
df = spark.read.format("csv").load("/content/Bank_final .csv")  # Changed "cvs" to "csv"

In [14]:
df.show(truncate=False)

+-----------+----------+---------+-------------+------------------------------+--------------+------------+-------+-----------+--------------+----------------+------+----------+
|_c0        |_c1       |_c2      |_c3          |_c4                           |_c5           |_c6         |_c7    |_c8        |_c9           |_c10            |_c11  |_c12      |
+-----------+----------+---------+-------------+------------------------------+--------------+------------+-------+-----------+--------------+----------------+------+----------+
|Customer ID|First Name|Last Name|Date of Birth|Email                         |Account Number|Account Type|Balance|Date Opened|Transaction ID|Transaction Type|Amount|Date      |
|6486       |Jasmine   |Thompson |1977-05-03   |dgarcia@example.com           |1225353200    |Checking    |7973.59|8/20/2023  |3154          |Transfer        |816.04|12-09-2022|
|4758       |Ronald    |Mccarty  |4/23/1998    |rita14@example.org            |5233596782    |Savings     |238

In [15]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)



In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# **Creating the datetime , Date**

In [22]:

from datetime import datetime, date
dtime = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
dtime

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [32]:
dtjson = spark.read.json("/content/1433379.json")

In [151]:
from re import MULTILINE
djson = spark.read.format("json").load("/content/1433379.json", MULTILINE = True)

In [152]:
from os import truncate
djson.show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------

In [153]:
djson.printSchema()

root
 |-- info: struct (nullable = true)
 |    |-- balls_per_over: long (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- dates: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- event: struct (nullable = true)
 |    |    |-- match_number: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- match_type: string (nullable = true)
 |    |-- match_type_number: long (nullable = true)
 |    |-- officials: struct (nullable = true)
 |    |    |-- match_referees: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- reserve_umpires: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- tv_umpires: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- umpires: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |   

In [60]:
djson.select("info.event").show(truncate = False)

+-----------------------------------+
|event                              |
+-----------------------------------+
|{2, Bangladesh tour of West Indies}|
+-----------------------------------+



In [62]:
from pyspark.sql.functions import col

In [64]:
djson.select(col("info.event")).show(truncate = False)


+-----------------------------------+
|event                              |
+-----------------------------------+
|{2, Bangladesh tour of West Indies}|
+-----------------------------------+



In [145]:
txt = spark.read.text("/content/sample_data/text1.txt")

In [146]:
txt.show(truncate = False)

+--------------------------------+
|value                           |
+--------------------------------+
|id: 101                         |
|name: Alice Johnson             |
|age: 28                         |
|email: alice.johnson@example.com|
|city: Los Angeles               |
|country: USA                    |
|profession: Data Scientist      |
|company: TechCorp               |
|salary: 95000                   |
|department: AI Research         |
|                                |
|id: 102                         |
|name: Bob Smith                 |
|age: 32                         |
|email: bob.smith@example.com    |
|city: New York                  |
|country: USA                    |
|profession: Software Engineer   |
|company: CodeWorks              |
|salary: 105000                  |
+--------------------------------+
only showing top 20 rows



In [147]:
from pyspark.sql.functions import explode, col, udf

In [150]:
# convert text to dictionary

In [148]:
from pyspark.sql.functions import udf
import ast
from pyspark.sql.types import MapType, StringType

# Define UDF to convert text to dictionary
def text_to_dict(text):
    pairs = [item.split(":", 1) for item in text.split(", ")] # Split by ':', but limit to 1 split to handle missing values
    return {k.strip(): v[0].strip() if len(v) > 0 else '' for item, (k, *v) in zip(text.split(", "), pairs)} # Assign empty string if value is missing and strip the first element if it's a list

# Register UDF with return type as MapType
text_to_dict_udf = udf(lambda z: text_to_dict(z), MapType(StringType(), StringType()))

In [149]:
df =txt.withColumn("data", text_to_dict_udf(col("value")))
df.show(truncate = False)

+--------------------------------+------------------------------------+
|value                           |data                                |
+--------------------------------+------------------------------------+
|id: 101                         |{id -> 101}                         |
|name: Alice Johnson             |{name -> Alice Johnson}             |
|age: 28                         |{age -> 28}                         |
|email: alice.johnson@example.com|{email -> alice.johnson@example.com}|
|city: Los Angeles               |{city -> Los Angeles}               |
|country: USA                    |{country -> USA}                    |
|profession: Data Scientist      |{profession -> Data Scientist}      |
|company: TechCorp               |{company -> TechCorp}               |
|salary: 95000                   |{salary -> 95000}                   |
|department: AI Research         |{department -> AI Research}         |
|                                |{ -> }                        

**bold text**

In [ ]:
df = df.withColumn(p)